In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from skimage.feature import hog, blob_dog
from skimage import data, exposure
from math import sqrt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from os import path

def hogfeatures2(base, fname):
    fpath = base + fname + ".jpeg" 
    image = np.array(Image.open(fpath), dtype="uint8")
    
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True, multichannel=True)
    
    hfc, hbin = np.histogram(fd, bins=64, range=(0.0, 1.0))
    hfp = hfc[1:] / np.sum(hfc[1:])
    
    return hfp

def dogfeatures2(base, fname):
    fpath0 = base + fname + ".jpeg" 
    image = np.array(Image.open(fpath0), dtype="uint8")
    
    image_gray = rgb2gray(image)
    blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.1)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
    
    image2 = np.zeros(image_gray.shape)
    
    for blob in blobs_dog:
        y, x, r = blob
        y1 = round(y - r)
        y2 = round(y + r)
        x1 = round(x - r)
        x2 = round(x + r)        
        image2[y1:y2, x1:x2] = 255
        
    image2 = image2 > 0     
    RGB = image[image2,:] 
    RGB = RGB / 255
    
    hfc, bin_edges = np.histogram(RGB[:, 0], bins=22, range=(0, 1)) 
    hfc2, bin_edges = np.histogram(RGB[:, 1], bins=22, range=(0, 1)) 
    hfc3, bin_edges = np.histogram(RGB[:, 2], bins=22, range=(0, 1)) 
    
    hfp = hfc[1:] / np.sum(hfc[1:])    
    hfp2 = hfc2[1:] / np.sum(hfc2[1:])    
    hfp3 = hfc3[1:] / np.sum(hfc3[1:])    
    
    hfp = np.concatenate((hfp, hfp2, hfp3), axis=0)
    
    return hfp

def readfeatures(cls, K):
    hfc1 = np.zeros([K, 63])
    hfc2 = np.zeros([K, 63])
    
    for num, fname in enumerate(cls['image'], start=0):            
        hfc1[num, :] = hogfeatures2("dbase/", fname)      
        hfc2[num, :] = dogfeatures2("dbase/", fname)    
        if num + 1 >= K:
            break
    
    hfc = np.concatenate((hfc1, hfc2), axis=1)
    
    return hfc

def normalize(vdata):     
    ncols = vdata.shape[1]  
    mx = np.zeros(ncols)
    mn = np.zeros(ncols) 
    for c in range(ncols): 
        mx[c] = np.max(vdata[:, c])
        mn[c] = np.min(vdata[:, c])
    
    vdata = (vdata - mn) / (mx - mn)    
    return vdata

def run_ml():
    cls = pd.read_csv("dbase/train.csv")
    
    K = cls.shape[0]
    
    hfc = readfeatures(cls, K)
    X = normalize(hfc)
    
    y = np.array(cls["label"])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    clf = MLPClassifier(hidden_layer_sizes=(100,), random_state=1, max_iter=300)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
    cm_display = ConfusionMatrixDisplay(cm, clf.classes_)
    
    plt.figure(figsize=(8, 8))
    cm_display.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()
    
    print("Accuracy:", accuracy)

run_ml()

KeyboardInterrupt: 